## manos

In [ ]:
import cv2
import mediapipe as mp

dibujo_mp = mp.solutions.drawing_utils
manos_mp = mp.solutions.hands

while(1):
    video = cv2.VideoCapture('../sources/video1.mp4')
    #video = cv2.VideoCapture(1)

    with manos_mp.Hands(
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        while video.isOpened():
            ret, frame = video.read()
            if not ret:

                break

            # Convertir la imagen de BGR a RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detección de manos

            results = hands.process(image)

            # Dibujar los landmarks de las manos en la imagen
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    dibujo_mp.draw_landmarks(
                        image, hand_landmarks, manos_mp.HAND_CONNECTIONS)

            # Mostrar la imagen resultante
            cv2.imshow('Video Procesado', image)
            cv2.imshow('Video Original', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    video.release()
    cv2.destroyAllWindows()

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks

  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image



import cv2

img = cv2.imread("../sources/girl.jpg")
cv2.imshow("imagen", img)


import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an PoseLandmarker object.
base_options = python.BaseOptions(model_asset_path='pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: Load the input image.

image = mp.Image.create_from_file("../sources/girl.jpg")

# STEP 4: Detect pose landmarks from the input image.
detection_result = detector.detect(image)

# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("imagen  1", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2.imshow("imagen  2", visualized_mask)


cv2.waitKey()
cv2.destroyAllWindows()

# cuerpo

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.framework.formats import landmark_pb2

dibujo_mp = mp.solutions.drawing_utils
cuerpos_mp = mp.solutions.pose

video = cv2.VideoCapture('../sources/video1.mp4')
#video = cv2.VideoCapture(4)
# Inicializar el objeto del detector de pose
#pose = cuerpos_mp.Pose()

with cuerpos_mp.Pose(model_complexity=1, min_tracking_confidence=0.5 ) as cuerpo:
    
    while video.isOpened():
        ret, frame = video.read()
        
        if not ret:
            break
        
        # Convertir la imagen de BGR a RGB
        imagen = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        # Detección de cuerpo
        resultado = cuerpo.process(imagen)

        # Dibujar los landmarks del cuerpo en la imagen
        imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2BGR)
        
        
        
        if resultado.pose_landmarks:
            
            '''
            #landmark = resultado.pose_landmarks
            
            # Obtener los puntos y coordenadas de las poses del cuerpo
            for cordenada in resultado.pose_landmarks.landmark:
                index = cordenada.landmark_index
                x = cordenada.x
                y = cordenada.y
                z = cordenada.z
                
                # Hacer algo con los puntos y coordenadas (ejemplo: imprimirlos)
                print(f"Punto {index}: Coordenadas (x, y, z) = ({x}, {y}, {z})")
            '''
            dibujo_mp.draw_landmarks(
                imagen, resultado.pose_landmarks, cuerpos_mp.POSE_CONNECTIONS)
        
        
        cv2.imshow("imagen", imagen)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.framework.formats import landmark_pb2

dibujo_mp = mp.solutions.drawing_utils
cuerpos_mp = mp.solutions.pose
manos_mp = mp.solutions.hands
rostro_mp = mp.solutions.face_mesh

video = cv2.VideoCapture('../sources/video1.mp4')


with cuerpos_mp.Pose(model_complexity=1, min_tracking_confidence=0.5 ) as cuerpo, \
manos_mp.Hands(
        max_num_hands=2,
        min_detection_confidence=0.5) as hands, \
rostro_mp.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5) as face_mesh:

    
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        
        # Convertir la imagen de BGR a RGB
        imagen = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        # Detección de cuerpo
        resultado = cuerpo.process(imagen)
        
        # Detección de manos
        results = hands.process(imagen)
        
        results2 = face_mesh.process(imagen)

        # Dibujar los landmarks del cuerpo en la imagen
        imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2BGR)
        
         

        
        if resultado.pose_landmarks:
            dibujo_mp.draw_landmarks(
                imagen, resultado.pose_landmarks, cuerpos_mp.POSE_CONNECTIONS)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                dibujo_mp.draw_landmarks(
                    imagen, hand_landmarks, manos_mp.HAND_CONNECTIONS)
        
        if results2.multi_face_landmarks:
            for face_landmarks in results2.multi_face_landmarks:
                dibujo_mp.draw_landmarks(
                    imagen, face_landmarks, rostro_mp.FACEMESH_LIPS   )

        
    
        
        cv2.imshow("imagen", imagen)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            

        
cv2.waitKey()
cv2.destroyAllWindows()